BBC, The Guardian, The Mirror, The Independent, The Times, The Telegraph, The Sun, The Daily Express, The Daily Mail


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
import pandas as pd

# BBC

## BBC Immigration

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 5

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(5, 14):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f"https://www.bbc.co.uk/news/topics/c302m85qe1vt?page={page_number}"
    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-immigration.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_immigration = pd.read_csv("bbc-uk-immigration.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# function to get body of article
def get_body(df, retries=3):
    bodies = []
    for link in df['Links']:
        for attempt in range(retries):
            try:
                res = requests.get(link, headers=headers)
                res.raise_for_status()  # raise an HTTPError for bad responses
                soup = bs(res.content, 'lxml')
                main_content = soup.find_all("div", {"class": "ssrcss-uf6wea-RichTextComponentWrapper ep2nwvo0"})

                # extract text from the text-block
                article_text = ' '.join(content.get_text() for content in main_content).strip()
                
                # append the body text
                bodies.append(article_text)
                break  # break the retry loop on success
            
            except (requests.exceptions.RequestException, requests.exceptions.SSLError) as e:
                print(f"Error fetching {link}: {e}")
                if attempt < retries - 1:
                    wait_time = 2 ** attempt  # exponential backoff
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to fetch {link} after {retries} attempts.")
                    bodies.append("")  # append an empty string if all retries fail

    # add the bodies list as a new column to the DataFrame
    df['Body'] = bodies

    return df

df = get_body(bbc_immigration)

# rewrite to CSV
df.to_csv(r"bbc-uk-immigration.csv", encoding="utf-8", header=True, index=False)



## BBC Migration

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 4

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(4, 16):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f"https://www.bbc.co.uk/news/topics/cz4pr2gdg1et?page={page_number}"

    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-migration.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_migration = pd.read_csv("bbc-uk-migration.csv")

df = get_body(bbc_migration)

# rewrite to CSV
df.to_csv(r"bbc-uk-migration.csv", encoding="utf-8", header=True, index=False)


## BBC Visas

In [ ]:
# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

# define our user headers
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
bbc = f"https://www.bbc.co.uk/news/topics/c1m1wly10gzt?page=1"

# request webpage
res = requests.get(bbc, headers=headers)

# check status for debugging
res.raise_for_status()

soup = bs(res.text, "html.parser")

# this gets list of articles
articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

for article in articles:
    # check if the article date contains the word "Video" or "Audio"
    article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
    if "Video" in article_date or "Audio" in article_date:
        continue  # Skip this article and move to the next one
    
    # append article link
    article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
    link = "https://www.bbc.co.uk" + article_info.attrs["href"]
    all_news_links.append(link)

    # append headline
    headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
    all_headlines.append(headline)

    # append date
    all_dates.append(article_date)

time.sleep(random.randint(1, 3))


# Convert data to dataframe
data = {"Headline": all_headlines,
    "Links": all_news_links,
    "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-visas.csv", encoding="utf-8", header="true", index=False)

In [ ]:
# reload csv
bbc_visas = pd.read_csv("bbc-uk-visas.csv")

df = get_body(bbc_visas)

# rewrite to CSV
df.to_csv(r"bbc-uk-visas.csv", encoding="utf-8", header=True, index=False)


## BBC Refugees and asylum seekers

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 9

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(9, 35):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f" https://www.bbc.co.uk/news/topics/cg41ylwvxmdt?page={page_number}"
    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-refugees.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_refugees = pd.read_csv("bbc-uk-refugees.csv")

df = get_body(bbc_refugees)

# rewrite to CSV
df.to_csv(r"bbc-uk-refugees.csv", encoding="utf-8", header=True, index=False)


## Merging all the dataframes


In [ ]:
# load csvs
bbc_refugees = pd.read_csv("bbc-uk-refugees.csv")
bbc_visas = pd.read_csv("bbc-uk-visas.csv")
bbc_migration = pd.read_csv("bbc-uk-migration.csv")
bbc_immigration = pd.read_csv("bbc-uk-immigration.csv")

# combine all dataframes
bbc = pd.concat([bbc_refugees, bbc_visas, bbc_migration, bbc_immigration])

# write to csv
bbc.to_csv(r"all-bbc.csv", encoding="utf-8", header=True, index=False)


# The Guardian

## Section: politics; Tag: uk/immigration

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/politics?tag=uk/immigration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-politics-uk-immigration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")



## Section: commentisfree; Tag: uk/immigration

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/commentisfree?tag=uk/immigration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-commentisfree-uk-immigration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


## Section: uk-news, Tag: global-development/migration



In [21]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/uk-news?tag=global-development/migration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-uk_news-global_development-migration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


Error: 400 - {"response":{"status":"error","message":"requested page is beyond the number of available pages"}}


## Section: uk-news; Tag: world/refugees

In [22]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/uk-news?tag=world/refugees&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-uk_news-world-refugees.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


Error: 400 - {"response":{"status":"error","message":"requested page is beyond the number of available pages"}}


## Merging all the dataframes

In [25]:
import pandas as pd
# load csvs
g1 = pd.read_csv("guardian-politics-uk-immigration.csv")
g2 = pd.read_csv("guardian-commentisfree-uk-immigration.csv")
g3 = pd.read_csv("guardian-uk_news-global_development-migration.csv")
g4 = pd.read_csv("guardian-uk_news-world-refugees.csv")

# combine all dataframes
guardian = pd.concat([g1, g2, g3, g4])

# write to csv
guardian.to_csv(r"all-guardian.csv", encoding="utf-8", header=True, index=False)


# Mirror

## Immigration

In [32]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 5):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/immigration"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/immigration?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-immigration.csv", encoding="utf-8", header="true", index=False)


We are scraping page: 1
We are scraping page: 2
We are scraping page: 3
We are scraping page: 4


In [31]:
import requests
from bs4 import BeautifulSoup as bs

# define the URL for the first page
mirror_url = "https://www.mirror.co.uk/all-about/immigration"

# define user headers
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# request webpage
res = requests.get(mirror_url, headers=headers)

# check status for debugging
res.raise_for_status()

# parse the HTML content
soup = bs(res.text, "html.parser")

# this gets list of articles
articles = soup.find_all("article")

# print out the href attribute of the <a> tag within each article that contains data-link-tracking
for article in articles:
    # find the <a> tag within the article that contains data-link-tracking
    a_tag = article.find('a', {"data-link-tracking": True})
    
    # extract the href attribute if the <a> tag is found
    if a_tag:
        href = a_tag.get('href')
        print(href)
    else:
        print("No link found")


https://www.mirror.co.uk/news/politics/ex-shopkeeper-local-legend-merseyside-32822286
https://www.mirror.co.uk/news/politics/britain-cant-address-illegal-migration-32784196
https://www.mirror.co.uk/news/politics/keir-starmer-vows-scrap-rwanda-32781412
https://www.mirror.co.uk/news/politics/keir-starmer-unveils-plan-replace-32774786
https://www.mirror.co.uk/news/politics/cruel-rwanda-scheme-re-opened-32692692
https://www.mirror.co.uk/news/politics/home-office-detain-asylum-seekers-32689199
https://www.mirror.co.uk/news/politics/dozens-small-boat-live-bbc-32648560
https://www.mirror.co.uk/news/uk-news/brexit-policy-tightening-borders-wont-31660930
https://www.mirror.co.uk/news/politics/stop-boats-vow-tatters-arrivals-32276845
https://www.mirror.co.uk/news/uk-news/one-charged-over-newhaven-swoop-32150331
https://www.mirror.co.uk/news/politics/uncle-tory-minister-tom-tugendhat-32108832
https://www.mirror.co.uk/news/uk-news/kids-returning-france-school-trip-32108956
https://www.mirror.co.uk